In [2]:
import sys
import json
import argparse
import pandas as pd
import numpy as np

sys.path.append('../../../')

import aizynthfinder.analysis
import aizynthfinder.chem
import aizynthfinder.context.config as con


from aizynthfinder.mcts.node import Node
from aizynthfinder.analysis import ReactionTree
from aizynthfinder.mcts.state import State
from aizynthfinder.chem import TreeMolecule, Molecule, UniqueMolecule
from aizynthfinder.context.collection import ContextCollection
from aizynthfinder.context.stock import StockException
import aizynthfinder.context.scoring as scoring

In [1]:
args_output = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/results/txt.txt'
args_input = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/price/results/json/tg1_393_3.json'
args_config = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/price/config_std.yml'

In [3]:
# create stock object
config = con.Configuration().from_file(args_config)
stock = config.stock

In [4]:
stock.price(UniqueMolecule(smiles="O=S(=O)(Cl)c1cccs1"))

Mol:  O=S(=O)(Cl)c1cccs1
SOMETHING
[]


StockException: Could not obtain price of molecule

In [3]:
with open(args_input) as inf:
    data = json.load(inf)

In [4]:

#solved reaction parse rxn tree
solved_reaction_data = []

print(len(jsonfile))

for p in range(len(jsonfile)-1):
    print('RUN: ', p)
    for i in jsonfile:
        if 'children' in i:
            child = i.get('children')[0]
            if child.get('type') == 'reaction':
                metadata = child.get('metadata')
                solved_reaction_data.append((child.get('smiles'), metadata.get('classification'), metadata.get('library_occurence'), metadata.get('policy_probability')))
            else:
                pass
        else:
            del jsonfile[-1]

smiles = [i[0] for i in solved_reaction_data]
classification = [i[1] for i in solved_reaction_data]
uspto_freq = [i[2] for i in solved_reaction_data]
policy = [i[3] for i in solved_reaction_data]


reaction_class = {}

for i in classification:
    if (i in reaction_class):
        reaction_class[i]['Frequency'] += 1
    else:
        reaction_class[i] = {}
        reaction_class[i]['Frequency'] = 1

reaction_class = dict(sorted(reaction_class.items(), key=lambda item: item[1]['Frequency'], reverse=True))

#calculate average policy predictions for each grouped reaction

reaction_class_policy = {}
reaction_class_freq = {}

for i in solved_reaction_data: 
    if i[1] in reaction_class_policy:
        reaction_class_policy[i[1]] += i[3]
        reaction_class_freq[i[1]] += 1
    else:
        reaction_class_policy[i[1]] = i[3]
        reaction_class_freq[i[1]] = 1

reaction_class_policy_mean = {}

for i in reaction_class_policy:
    reaction_class_policy_mean[i] = (reaction_class_policy.get(i)/reaction_class_freq.get(i))

#reaction_class_policy_mean = dict(sorted(reaction_class_policy_mean.items(), key=lambda item: item[1], reverse=True))
print('\n')
print('Rank, Reaction, Frequency, Policy value')

'''for x in reaction_class_policy_mean:
    print (x + '    '+ str(reaction_class_policy_mean[x]))'''

for i in reaction_class:
    reaction_class[i]['Policy'] = reaction_class_policy_mean.get(i)

count = 0
for i in reaction_class:
    print(str(count)+','+i+', '+str(reaction_class[i]['Frequency'])+', '+str(reaction_class[i]['Policy']))
    count+=1

reaction_class_lit = {}
reaction_class_freq = {}

for i in solved_reaction_data: 
    if i[1] in reaction_class_lit:
        reaction_class_lit[i[1]] += i[2]
        reaction_class_freq[i[1]] += 1
    else:
        reaction_class_lit[i[1]] = i[2]
        reaction_class_freq[i[1]] = 1

reaction_class_lit_mean = {}


for i in reaction_class_lit:
    reaction_class_lit_mean[i] = (reaction_class_lit.get(i)/reaction_class_freq.get(i))

reaction_class_lit_mean = dict(sorted(reaction_class_lit_mean.items(), key=lambda item: item[1], reverse=True))

print('Rank, Reaction, Frequency, Policy, Literature')
#f.write('\n\nRank, Reaction, Frequency, Policy, Literature\n')

'''for x in reaction_class_lit_mean:
    print (x + '    '+ str(reaction_class_lit_mean[x]))'''


for i in reaction_class:
    reaction_class[i]['Literature'] = reaction_class_lit_mean.get(i)

count = 1
for i in reaction_class:
    print(str(count)+','+i+', '+str(reaction_class[i]['Frequency'])+', '+str(round(reaction_class[i]['Policy'], 4))+', '+str(round(reaction_class[i]['Literature'], 0)))
    #f.write(str(count)+','+i+', '+str(reaction_class[i]['Frequency'])+', '+str(round(reaction_class[i]['Policy'], 4))+', '+str(round(reaction_class[i]['Literature'], 0))+'\n')
    count+=1


# route cost calculator
config = con.Configuration()
config = config.from_file(args_config)

rxns = [ReactionTree.from_dict(tree) for i in jsonfile for x, tree in enumerate(i)]


policy_scorer = scoring.USPTOModelPolicyProbabilityScorer()
policy_scores = policy_scorer(rxns)
policy_mean = np.mean(policy_scores)

cost_scorer = scoring.RouteCostScorer(config=config)
cost_scores = cost_scorer(rxns)
cost_mean = np.mean(cost_scores)

num_reactions_scorer = scoring.NumberOfReactionsScorer()
num_reactions_scores = num_reactions_scorer(rxns)
num_reactions_mean = np.mean(num_reactions_scores)

overall_cost = (0.2*100 - (1/10)*100)/num_reactions_mean

print('policy: ', policy_mean)
#f.write('\n\npolicy: '+ str(policy_mean)+'\n')

print('Cost: ', cost_mean)
#f.write('Cost: '+ str(cost_mean)+'\n')

print('Cost Term: ', cost_mean/100)

print('Num reactions: ', num_reactions_mean)
#f.write('Num Reactions: '+ str(num_reactions_mean)+'\n')

print('Target Library Synthesis Cost: ', cost_mean)
#f.write('Target Lib cost: '+ str(cost_mean)+'\n')
#f.close()

5
RUN:  0
RUN:  1
RUN:  2
RUN:  3


Rank, Reaction, Frequency, Policy value
0,Amide to amine reduction, 12, 0.005499999970197678
1,Heteroaryl N-alkylation, 8, 0.0007999999797903001
Rank, Reaction, Frequency, Policy, Literature
1,Amide to amine reduction, 12, 0.0055, 1676.0
2,Heteroaryl N-alkylation, 8, 0.0008, 39.0


TypeError: string indices must be integers

In [7]:
def get_all_reactions(nested_dict, in_list):
    all_reactions = []
    for key, value in nested_dict.items():
        #print('Key: ', key)
        #print('Value', value)
        if value == 'mol':
            #print('mol found')
            children  = nested_dict.get('children')
            for i in children:
                if 'children' in i:
                    get_all_reactions(nested_dict=i, in)
        elif value == 'reaction':
            #print('reaction found')
            metadata = nested_dict.get('metadata')
            #print(metadata)
            all_reactions.append((metadata.get('classification'), metadata.get('library_occurence'), metadata.get('policy_probability')))
            children  = nested_dict.get('children')
            for i in children:
                if 'children' in i:
                    get_all_reactions(nested_dict=i)
        elif (key == 'is_stock' and value=='True'):
            #print('REACHED STOCK')
            pass
        else:
            continue
    return in_list+all_reactions

In [4]:
for i in jsonfile:
    results = get_all_reactions(i)

NameError: name 'jsonfile' is not defined

In [4]:
rxns = [ReactionTree.from_dict(i) for i in data]

In [5]:
print(len(rxns))
print(type(rxns[1]))
print(rxns)

6
<class 'aizynthfinder.analysis.ReactionTree'>
[<aizynthfinder.analysis.ReactionTree object at 0x7f74249788b0>, <aizynthfinder.analysis.ReactionTree object at 0x7f7424978940>, <aizynthfinder.analysis.ReactionTree object at 0x7f74249782e0>, <aizynthfinder.analysis.ReactionTree object at 0x7f7424978730>, <aizynthfinder.analysis.ReactionTree object at 0x7f7424946130>, <aizynthfinder.analysis.ReactionTree object at 0x7f7424946670>]


In [16]:
config = con.Configuration()
config = config.from_file(args_config)

In [17]:
stock = config.stock

In [18]:
instock = [i for i in list(leaves) if i in stock]

NameError: name 'leaves' is not defined

In [22]:
mol = Molecule(smiles="CC(=O)Cl")
mol.inchi_key

'WETWJCDKMRHUPV-UHFFFAOYSA-N'

In [21]:
if mol in stock:
    print('YES')
else: 
    print('NO')

NO


In [ ]:
tree = rxns[-1]
leaves = tree.leafs()
print(list(leaves))

In [23]:
tree = rxns[-1]
leaves = tree.
print(list(leaves))
for i in list(leaves):
    if i in stock:
        print('in')
    else:
        print('out')

SyntaxError: invalid syntax (<ipython-input-23-2189c1f430d4>, line 2)

In [15]:
cost_scorer = scoring.PriceSumScorer(config=config)
cost_scoes = cost_scorer(rxns)
print(cost_scoes)

test
mol:  (0, <aizynthfinder.chem.UniqueMolecule object at 0x7f996c4ead30>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
test
mol:  (0, <aizynthfinder.chem.UniqueMolecule object at 0x7f9900442040>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
mol:  (1, <aizynthfinder.chem.UniqueMolecule object at 0x7f9900442280>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
mol:  (2, <aizynthfinder.chem.UniqueMolecule object at 0x7f99004421c0>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
mol:  (3, <aizynthfinder.chem.UniqueMolecule object at 0x7f996c5155e0>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
test
mol:  (0, <aizynthfinder.chem.UniqueMolecule object at 0x7f9900442700>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
mol:  (1, <aizynthfinder.chem.UniqueMolecule object at 0x7f9900442790>)
type:  <class 'aizynthfinder.chem.UniqueMolecule'>
mol not in stock
mol: 